In [ ]:
# pip install vtk
import os
from tqdm import tqdm
import vtk
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy as vtk2np
import glob

ModuleNotFoundError: No module named 'vtk'

In [2]:
def read_vtm_file(file_path):
    # Redirect VTK messages to the console
    ow = vtk.vtkFileOutputWindow()
    ow.SetFileName("vtk_errors.log")
    vtk.vtkOutputWindow().SetInstance(ow)
    # Create a reader for the .vtm file
    reader = vtk.vtkXMLMultiBlockDataReader()
    reader.SetFileName(file_path)
    reader.Update()

    # Get the output of the reader
    multiblock = reader.GetOutput()

    data_blocks = []
    grid_positions = []

    # Iterate through each block (corresponds to each .vtr file)
    for i in range(multiblock.GetNumberOfBlocks()):
        block = multiblock.GetBlock(i)
        if block is not None:
            block_data, position = process_vtr_data(block)
            if block_data is not None:
                data_blocks.append(block_data)
                grid_positions.append(position)

    return data_blocks, grid_positions


# Function to process individual .vtr block
def process_vtr_data(rectilinear_grid: vtk.vtkRectilinearGrid):
    # Get the coordinates
    x_coords = vtk_to_numpy(rectilinear_grid.GetXCoordinates())
    y_coords = vtk_to_numpy(rectilinear_grid.GetYCoordinates())
    z_coords = vtk_to_numpy(rectilinear_grid.GetZCoordinates())
    # Get the point data
    point_data = rectilinear_grid.GetPointData()
    num_arrays = point_data.GetNumberOfArrays()
    data_arrays = []
    for i in range(num_arrays):
        array = vtk_to_numpy(point_data.GetArray(i))
        # Adjust the reshape to account for z being the fastest-changing axis
        array = array.reshape(
            (x_coords.shape[0], y_coords.shape[0], z_coords.shape[0]), order="F"
        )
        data_arrays.append(array)
    i = 1
    data_4d_array = np.stack(data_arrays, axis=0)[:, i:-i, i:-i, i:-i]
    return data_4d_array, (x_coords[i:-i], y_coords[i:-i], z_coords[i:-i])


# Helper function to convert VTK arrays to numpy arrays
def vtk_to_numpy(vtk_array):
    return vtk2np(vtk_array)


def merge_blocks(data_blocks, grid_positions):
    # Flatten and combine all coordinates along each axis
    x_coords = np.concatenate([pos[0].flatten() for pos in grid_positions])
    y_coords = np.concatenate([pos[1].flatten() for pos in grid_positions])
    z_coords = np.concatenate([pos[2].flatten() for pos in grid_positions])

    # Find unique global coordinates
    x_unique = np.unique(x_coords)
    y_unique = np.unique(y_coords)
    z_unique = np.unique(z_coords)

    # Determine the number of data arrays from the first block
    num_arrays = data_blocks[0].shape[0]

    # Create empty global domain array with an extra dimension for the data arrays
    global_shape = (num_arrays, len(x_unique), len(y_unique), len(z_unique))
    global_array = np.full(global_shape, np.nan)  # Use NaN to identify unfilled regions

    # Create lookup for indices
    x_index = {val: i for i, val in enumerate(x_unique)}
    y_index = {val: i for i, val in enumerate(y_unique)}
    z_index = {val: i for i, val in enumerate(z_unique)}

    # Map each block to the global array
    for block, pos in zip(data_blocks, grid_positions):
        x, y, z = pos
        x_flat = x.flatten()
        y_flat = y.flatten()
        z_flat = z.flatten()

        x_idx = [x_index[val] for val in x_flat]
        y_idx = [y_index[val] for val in y_flat]
        z_idx = [z_index[val] for val in z_flat]

        # Create slices for the block data
        x_slice = slice(min(x_idx), max(x_idx) + 1)
        y_slice = slice(min(y_idx), max(y_idx) + 1)
        z_slice = slice(min(z_idx), max(z_idx) + 1)

        # Insert each data array into the corresponding slice of the global array
        for i in range(num_arrays):
            global_array[i, x_slice, y_slice, z_slice] = block[i]

    return global_array, (x_unique, y_unique, z_unique)

In [3]:
# Change the path based on where you store it
folder_path="/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/"
vtm_files=glob.glob(f"{folder_path}/*0.vtm")+ glob.glob(f"{folder_path}/*5.vtm")
vtm_files=sorted(vtm_files, key=lambda x: int(x.split('channel.')[1].split('.')[0]))
vtm_files

['/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/channel.234815.vtm',
 '/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/channel.234820.vtm',
 '/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/channel.234825.vtm',
 '/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/channel.234830.vtm',
 '/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/channel.234835.vtm',
 '/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/channel.234840.vtm',
 '/home/leo/serverMnt/server/projects/1051-TOTAL-DNS-Roughness-Non-Newtonian/14-Data-Analytics/01-NS-Mesh3-restart/RESULT/channel.234845.vtm',

In [4]:
import os
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# Define the function to process a single file
def process_vtm_file(vtm_file_path):
    i = vtm_file_path.split("channel.")[-1].split(".vtm")[0]
    # Change the path based on where you use it
    file_path = f"/mnt/c/Users/qcl656/OneDrive - AFRY/Documents/PROJECTS/Total_G-LED/data/NS_mesh3_full_simulation/global_domain_float16_{i}.npy"
    if os.path.exists(file_path):
        return
    data_blocks, grid_positions = read_vtm_file(vtm_file_path)
    global_domain, coordinates = merge_blocks(data_blocks, grid_positions)
    global_domain = global_domain[2:5].astype(np.float16)
    np.save(
        file_path,
        global_domain.astype(np.float16),
    )

# Use ProcessPoolExecutor for parallel processing
def parallel_process_vtm_files(vtm_files, max_workers=4):
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(process_vtm_file, vtm_files), total=len(vtm_files)))

parallel_process_vtm_files(vtm_files, max_workers=8)


100%|██████████| 12039/12039 [00:03<00:00, 3880.31it/s]


# Combine timesteps

In [3]:
import numpy as np
import glob
import os

# List of .npy files
folder_path = "../data/NS_mesh3_full_simulation/"

# Get data every 5 timestemps
npy_files = [
    os.path.join(folder_path, f)
    for f in os.listdir(folder_path)
    if (f.endswith("5.npy") or f.endswith("0.npy"))
]
npy_files = sorted(npy_files, key=lambda x: int(x.split("float16_")[1].split(".")[0]))
len(npy_files)

100

In [5]:

# Parameters
batch_size = 30
# output_dir = '../data/global_domain_float16_double'  # Directory to save combined batches
output_dir = '../data/batches'  # Directory to save combined batches

os.makedirs(output_dir, exist_ok=True)  # Create directory if not exists

# Process files in batches
for i in range(0, len(npy_files), batch_size):
    batch_files = npy_files[i:i+batch_size]
    arrays = [np.load(f) for f in batch_files]  # Load arrays
    combined_batch = np.stack(arrays, axis=0)  # Add new dimension and stack
    
    # Save the combined batch
    batch_filename = os.path.join(output_dir, f'combined_batch_{i // batch_size}.npy')
    np.save(batch_filename, combined_batch)
    print(f"Saved: {batch_filename}")


Saved: ../data/batches/combined_batch_0.npy
Saved: ../data/batches/combined_batch_1.npy
Saved: ../data/batches/combined_batch_2.npy
Saved: ../data/batches/combined_batch_3.npy
